In [6]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from sklearn.metrics import accuracy_score

from torch.utils.data import Dataset, DataLoader,TensorDataset
import pandas as pd
import time
from torchsummary import summary
from tqdm import tqdm

from torch.utils.data import Subset #

 **数据加载** 

In [7]:
# 将图像转化为pytorch张量
transform = transforms.Compose([transforms.ToTensor()])
"BATCH_SIZE = 128"
BATCH_SIZE = 64
# 加载训练集
ds_train = torchvision.datasets.ImageFolder(root="./data/tiny-imagenet-200/train",  transform=transform)
# 加载验证集
ds_valid = torchvision.datasets.ImageFolder(root="./data/tiny-imagenet-200/val/", transform=transform)

# 仅用一部分
# 选择前1000个样本
indices = list(range(1000))
ds_train = Subset(ds_train, indices)
indices = list(range(1000))
ds_valid = Subset(ds_valid, indices)

# 创建DataLoader对象来批量加载数据
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
"""
shuffle = True：打乱顺序
num_workers = 0：不使用多线程
"""
print("已完成数据加载")

已完成数据加载


In [8]:
import models.densenet
model = models.densenet.densenet121()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DenseNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (features): Sequential(
    (dense_block_layer_0): Sequential(
      (bottle_neck_layer_0): Bottleneck(
        (bottle_neck): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU(inplace=True)
          (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bottle_neck_layer_1): Bottleneck(
        (bottle_neck): Sequential(
          (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU(inplace=True)
          (2): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(12

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    start = time.time()
    model.train()
    sun_loss = 0
    sum_correct = 0
    
    # 使用 tqdm 包装 train_loader 以显示进度条
    for step, (data, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}", unit="batch")):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions = model(data)
        loss = model.loss_func(predictions, labels)
        loss.backward()
        optimizer.step()
        sun_loss += loss.item()
        _, preds = predictions.max(1)
        sum_correct += preds.eq(labels).sum()

        # 每 100 步输出一次训练进度
        if (step % 100 == 0) and (step != 0):
            print(f'Train Epoch: {epoch} [{step * len(data)}/{len(train_loader.dataset)} ({100. * step / len(train_loader):.0f}%)]\tTrain Loss: {loss.item() / BATCH_SIZE:.6f}')
    
    finish = time.time()
    print(f'Epoch {epoch} training time consumed: {finish - start:.2f}s')
    print(f'Train set: Epoch: {epoch}, Train loss: {sun_loss / len(train_loader.dataset):.4f}, Train Accuracy: {sum_correct.float() / len(train_loader.dataset):.4f}')

def test(model, device, test_loader, epoch):
    start = time.time()
    model.eval()
    test_loss = 0
    correct = 0
    step = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = model.loss_func(outputs, labels)
            test_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum()
            step += 1

    finish = time.time()
    print('Test set: Epoch: {}, Val loss: {:.4f}, Val Accuracy: {:.4f}, Time consumed:{:.2f}s'.format(
        epoch,
        test_loss / len(test_loader.dataset),
        correct.float() / len(test_loader.dataset),
        finish - start
    ))
    acc = correct.float() / len(test_loader.dataset)
    return acc

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.loss_func = nn.CrossEntropyLoss()
EPOCHS = 100
model.to(device)

for epoch in range(1, EPOCHS + 1):
    print(epoch)
    train(model, device, dl_train, optimizer, epoch)
    acc = test(model, device, dl_valid, epoch)

acc = acc.cpu()
acc = round(float(acc), 5)
torch.save(model, "./tiny_dense101_acc_" + str(acc) + "_.pt")

1


Epoch 1: 100%|██████████████████████████████| 16/16 [02:08<00:00,  8.01s/batch]


Epoch 1 training time consumed: 128.12s
Train set: Epoch: 1, Train loss: 0.0175, Train Accuracy: 0.8130
Test set: Epoch: 1, Val loss: 0.0780, Val Accuracy: 0.1330, Time consumed:35.49s
2


Epoch 2: 100%|██████████████████████████████| 16/16 [02:06<00:00,  7.88s/batch]


Epoch 2 training time consumed: 126.11s
Train set: Epoch: 2, Train loss: 0.0033, Train Accuracy: 0.9440
Test set: Epoch: 2, Val loss: 0.0122, Val Accuracy: 0.7410, Time consumed:35.68s
3


...

Epoch 14: 100%|█████████████████████████████| 16/16 [02:08<00:00,  8.05s/batch]


Epoch 14 training time consumed: 128.80s
Train set: Epoch: 14, Train loss: 0.0019, Train Accuracy: 0.9560
Test set: Epoch: 14, Val loss: 0.0610, Val Accuracy: 0.2360, Time consumed:35.40s
15


Epoch 15: 100%|█████████████████████████████| 16/16 [02:06<00:00,  7.88s/batch]


Epoch 15 training time consumed: 126.05s
Train set: Epoch: 15, Train loss: 0.0015, Train Accuracy: 0.9670
Test set: Epoch: 15, Val loss: 0.0332, Val Accuracy: 0.4560, Time consumed:35.27s
16


Epoch 16:   6%|█▉                            | 1/16 [00:07<01:58,  7.90s/batch]